In [2]:
import json
import os
import numpy as np
import pandas as pd

from utils import apply_sliding_window, label_dict, convert_labels_to_annotation_json

total_sbjs = ['sbj_1', 'sbj_2', 'sbj_3', 'sbj_4']

#define split for validation (kepping subject 1 in training for 3 splits and 2,3,4
#  acts as validation for each split)
sbj_split = [['sbj_2'], ['sbj_3'], ['sbj_4']]

# change these parameters
window_overlap = 50 # In percentage
window_size = 30 #for 1s window frame
# sampling frequency is 30 Hz (for 0.5s (Window size = 15), 1s (Window size = 30), 2s (Window size = 60))
frames = 60 #assuming 60fps
stride = 30 # 30 strides for 1 second frame

# change output folder
raw_inertial_folder = './opportunity_adl_data/raw_filled_zero/'
inertial_folder = './opportunity_adl_data/concatenated_data/inertial_features/{}_frames_{}_stride'.format(frames, stride)
os.makedirs(inertial_folder, exist_ok=True)

# fixed dataset properties
nb_sbjs = 4
fps = 60
sampling_rate = 30

for sbj in total_sbjs:
    raw_inertial_sbj = pd.read_csv(os.path.join(raw_inertial_folder, sbj + '.csv'), index_col=None)          
    inertial_sbj = raw_inertial_sbj.replace({"label": label_dict}).fillna(-1).to_numpy()
    _, win_sbj, _ = apply_sliding_window(inertial_sbj, window_size, window_overlap)
    flipped_sbj = np.transpose(win_sbj[:, :, 1:], (0,2,1))
    flat_win_sbj = win_sbj.reshape(win_sbj.shape[0], -1)
    output_inertial = flipped_sbj.reshape(flipped_sbj.shape[0], -1)
    np.save(os.path.join(inertial_folder, sbj + '.npy'), output_inertial)

for i, split_sbjs in enumerate(sbj_split):
    opportunity_annotations = {'version': 'opportunity', 'database': {}, 'label_dict': label_dict}
    # create video annotations
    for j in range(1,nb_sbjs+1):
        curr_sbj = "sbj_"+str(j)
        raw_inertial_sbj_t = pd.read_csv(os.path.join(raw_inertial_folder, curr_sbj + '.csv'), index_col=None)
        duration_seconds = len(raw_inertial_sbj_t) / sampling_rate
        sbj_annos = convert_labels_to_annotation_json(raw_inertial_sbj_t.iloc[:, -1], sampling_rate, fps, label_dict)
        if curr_sbj in split_sbjs:
            train_test = 'Validation'
        else:
            train_test = 'Training'
        opportunity_annotations['database']['sbj_' + str(int(j))] = {
            'subset': train_test,
            'duration': duration_seconds,
            'fps': fps,
            'annotations': sbj_annos,
            } 
        with open('./' + 'opportunity_split_' + str(int(i+1)) +  '.json', 'w') as outfile:
            outfile.write(json.dumps(opportunity_annotations, indent = 4))


/var/folders/5d/l0hr0c0n41326gh3kfwrj42c0000gn/T/ipykernel_34190/3917380959.py:32: DtypeWarning: Columns (114) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_inertial_sbj = pd.read_csv(os.path.join(raw_inertial_folder, sbj + '.csv'), index_col=None)
/var/folders/5d/l0hr0c0n41326gh3kfwrj42c0000gn/T/ipykernel_34190/3917380959.py:32: DtypeWarning: Columns (114) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_inertial_sbj = pd.read_csv(os.path.join(raw_inertial_folder, sbj + '.csv'), index_col=None)
/var/folders/5d/l0hr0c0n41326gh3kfwrj42c0000gn/T/ipykernel_34190/3917380959.py:45: DtypeWarning: Columns (114) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_inertial_sbj_t = pd.read_csv(os.path.join(raw_inertial_folder, curr_sbj + '.csv'), index_col=None)
/var/folders/5d/l0hr0c0n41326gh3kfwrj42c0000gn/T/ipykernel_34190/3917380959.py:45: DtypeWarning: Columns (114) have mixed types. Specify dtype o